# CSC 210 Project V1
## 1. Scrape tweets, create db

In [18]:
### This script takes 
import keys
import json
import tweepy
import pandas as pd
from sentiment import *

def users_tweets(user):
    tweets_frame = get_tweets(user)
    counter = 30
    while counter != 0:
        id = tweets_frame.iloc[-1]["Tweet_ID"]
        tweets_frame = tweets_frame.append(get_tweets(user, id))
        counter -= 1
    tweets_frame.reset_index(inplace=True, drop=True)
    return tweets_frame

def save_tweets(user_list, polarity):
    for user in user_list:
        tweets_frame = users_tweets(user)
        if polarity == "lib":
            tweets_frame.to_json(path_or_buf=f"./ScrapedTweets/Libs/{user}_libtweets.json", orient="records")
            print(f"Created {user}_libtweets.json")
        else: 
            tweets_frame.to_json(path_or_buf=f"./ScrapedTweets/Cons/{user}_contweets.json", orient="records", force_ascii=True)
            #tweets_frame.to_csv(path_or_buf=f"./ScrapedTweets/Cons/{user}_scrapedtweets.csv")
            print(f"Created {user}_contweets.json \n")
      

def main():
    user_list = [
#        "benshapiro", 
#        "RandPaul", 
#        "TPUSA", 
#        "EricTrump", 
#        "QuinHillyer", 
#        "KimStrassel", 
#        "marklevinshow", 
#        "brad_polumbo", 
#        "EliseStefanik", 
#        "kayleighmcenany", 
#        "RealRLimbaugh", 
        "GOPChairwoman",
        "SteveGuest", 
        "AlexMarlow", 
        "TianaTheFirst"]
    polarity = input("Lib/Con?").lower()
    save_tweets(user_list, polarity)
main()

Created GOPChairwoman_contweets.json 

Created SteveGuest_contweets.json 

Created AlexMarlow_contweets.json 

Created TianaTheFirst_contweets.json 



In [6]:
from os import listdir
import pandas as pd


def big_json(small_path, big_path, file_name):
    big_frame = pd.DataFrame(columns = ["Tweet_ID", "Text", "Date", "User"])
    # This line is stolen! I kept getting a permission error using listdir. Apparently this is the most pythonic? - Google Stack Overflow 3207219 for reference
    files = (_, _, filenames) = next(os.walk(small_path))

    for file in files[2]:
        big_frame = big_frame.append(pd.read_json(f"{small_path}/{file}", orient="records"))

    big_frame.reset_index(inplace=True, drop=True)
    big_frame.to_json(path_or_buf=big_path, orient="records", force_ascii=True)

def main():
    file_name = "cons_big"
    big_path = f"./ScrapedTweets/Compiled/{file_name}.json"
    small_path = "./ScrapedTweets/Cons"
    big_json(small_path, big_path, file_name)
main()

## 2. Create Training Set

## Plotting with BokehJS

In [7]:
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import ColumnDataSource, Span as span
from bokeh.plotting import figure
from bokeh.models import Legend, LegendItem


def hyp_line(inv):
    """Create a line to model the hypothetical trend, based off of nothing really"""
    # Create x values with random decimals within range, reshape too
    x = np.linspace(-6,6,100).reshape(100, 1)
    
    # Check inverse flag (For a search favorable for conservatives, the sin function would be reversed)
    if inv == True:
        # Create y variables using this function, reshape too
        y = np.sin(-x/-3.819).reshape(100, 1)
    else:
        # Create y variables using this function, inverse though
        y = np.sin(x/3.819).reshape(100, 1)
        
    # Create pandas data frame by combining arrays and assigning columns (combined array is not stored as new variable to save a line)
    df = pd.DataFrame(data=np.concatenate((x,y), axis=1), columns=["x", "y"])
    
    # Set source and create line variable
    source = ColumnDataSource(df)
    line = p.line(x="x", y="y", source=source, line_width=2, line_alpha=.5, legend_label="Hypothetical Trend")
    
    # Return line
    return line

def style_plot():
    """Style plot"""
    # Create/show axes
    y_axis = span(location=0,dimension='height', line_color="darkslategray", line_dash='dashed', line_width=2)
    x_axis = span(location=0,dimension='width', line_color="darkslategray", line_dash='dashed', line_width=2)
    p.add_layout(y_axis)
    p.add_layout(x_axis)
    
    # Create/show polarity axes
    con_1 = span(location=-2, dimension="height", line_color="darkslategray", line_dash="solid", line_width=.5)
    con_2 = span(location=-4, dimension="height", line_color="darkslategray", line_dash="solid", line_width=.5)
    lib_1 = span(location=2, dimension="height", line_color="darkslategray", line_dash="solid", line_width=.5)
    lib_2 = span(location=4, dimension="height", line_color="darkslategray", line_dash="solid", line_width=.5)
    p.add_layout(con_1)
    p.add_layout(con_2)
    p.add_layout(lib_1)
    p.add_layout(lib_2)
    
    # Hide normal axes/ticks
    p.xaxis.axis_line_width = 0
    p.yaxis.axis_line_width = 0
    p.xgrid.grid_line_width = 0
    p.ygrid.grid_line_width = 0
    p.xaxis.minor_tick_line_color = None
    p.yaxis.minor_tick_line_color = None
    
    # Labels
    p.xaxis.axis_label = "Partisan Leanings (Anecdotally)"
    p.yaxis.axis_label = "Sentiment"   
    p.title.text_font_size = "18px"

    # Legend
    p.legend.location = "top_left"
    p.legend.click_policy="hide"

def show_notebook():
    show(p)
    output_notebook()
    
def create_plot(term):
    """Create global p to be used by all functions"""
    global p
    p = figure(plot_width=720, plot_height=480, x_range=(-6,6), y_range=(-1,1), title=f"Sentiment of Tweets About '{term.capitalize()}' and Partisan Leanings")
    
    
def main():
    term = "Term"
    # Create plot
    create_plot(term)
    
    # Objects to draw
    hyp_line(inv=False)
    
    # Style plot and add legend
    style_plot()
    
    #Show notebook 
    show_notebook()
main()

Loading BokehJS ...